In [20]:
list_of_packages <- c(
       "ggplot2",
       "dplyr",
       "tidyr",
       "ComplexHeatmap",
       "tibble",
       "RColorBrewer",
       "scales",
       "circlize"
)
for (package in list_of_packages) {
    suppressPackageStartupMessages(
        suppressWarnings(
            library(
                package,
                character.only = TRUE,
                quietly = TRUE,
                warn.conflicts = FALSE
            )
        )
    )
}
#

ERROR: Error in library(package, character.only = TRUE, quietly = TRUE, warn.conflicts = FALSE): there is no package called ‘ComplexHeatmap’


In [13]:
# Get the current working directory and find Git root
find_git_root <- function() {
    # Get current working directory
    cwd <- getwd()
    
    # Check if current directory has .git
    if (dir.exists(file.path(cwd, ".git"))) {
        return(cwd)
    }
    
    # If not, search parent directories
    current_path <- cwd
    while (dirname(current_path) != current_path) {  # While not at root
        parent_path <- dirname(current_path)
        if (dir.exists(file.path(parent_path, ".git"))) {
            return(parent_path)
        }
        current_path <- parent_path
    }
    
    # If no Git root found, stop with error
    stop("No Git root directory found.")
}

# Find the Git root directory
root_dir <- find_git_root()
cat("Git root directory:", root_dir, "\n")

Git root directory: /home/lippincm/Documents/GFF_3D_organoid_profiling_pipeline 


In [14]:
figures_path <- file.path(root_dir,"5.EDA/figures/")
if (!dir.exists(figures_path)) {
  dir.create(figures_path, recursive = TRUE)
}

In [18]:
organoid_consensus_df <- arrow::read_parquet(file.path(root_dir,"data/all_patient_profiles/organoid_consensus_profiles_annotated_drugs.parquet"))
# drop the therapeautic catagorty column
organoid_consensus_df <- organoid_consensus_df %>%
  select(-`Therapeutic Categories`)
head(organoid_consensus_df)

patient,treatment,dose,unit,Colocalization_Organoid_AGP.BF_MIN.CORRELATION.COEFF,Colocalization_Organoid_AGP.BF_MEDIAN.MANDERS.COEFF.M1,Colocalization_Organoid_AGP.BF_MAX.MANDERS.COEFF.M2,Colocalization_Organoid_AGP.BF_MAX.OVERLAP.COEFF,Colocalization_Organoid_AGP.BF_MIN.K1,Colocalization_Organoid_AGP.BF_MIN.K2,⋯,Texture_Organoid_ER_Sum.Average_256.3,Texture_Organoid_ER_Variance_256.3,Texture_Organoid_Mito_Contrast_256.3,Texture_Organoid_Mito_Sum.Average_256.3,Texture_Organoid_Mito_Variance_256.3,Area.Size.Shape_Organoid_EXTENT,Area.Size.Shape_Organoid_EULER.NUMBER,Area.Size.Shape_Organoid_SURFACE.AREA,Target,Class
<chr>,<chr>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
NF0014,Binimetinib,1,uM,0.2501474,0.2500005,-0.95896255,-0.93358445,-0.4466027,0.73477984,⋯,-1.1548391,-0.9579401,0.05826345,-0.5136133,-0.1974620,-0.75341735,-0.2960704,-0.7737309,MEK1/2 inhibitor,Small Molecule
NF0014,Binimetinib,10,uM,-0.5282621,0.2500012,0.08586530,-0.96028678,-0.4676706,0.91788047,⋯,-1.0816036,-0.9025234,-0.71989968,-0.8146268,-0.3414513,0.74337630,-0.4088304,-1.3673196,MEK1/2 inhibitor,Small Molecule
NF0014,Cabozantinib,1,uM,0.1597416,0.2500012,-0.64064094,-0.86665756,-0.3440249,0.08570623,⋯,-0.6857140,-0.6729403,-0.31720371,-0.4620224,-0.1336881,0.17501886,-0.4088304,-1.0556796,receptor tyrosine kinase inhibitor,Small Molecule
NF0014,Copanlisib,1,uM,0.2604506,0.2499975,-1.06708032,-0.95006171,-0.5071072,0.36197497,⋯,-0.9617208,-0.7605898,0.53353062,-0.6605660,-0.3063693,-0.42659080,-0.3883285,-1.1374567,PI3K inhibitor,Small Molecule
NF0014,DMSO,1,%,0.4938911,0.2500007,0.09903665,0.06249213,-0.3566118,-0.12649450,⋯,-0.2914359,-0.3589357,-0.30694942,-0.2155743,-0.4293686,-0.07543949,-0.4088304,-0.4588865,Control,Control
NF0014,Digoxin,1,uM,0.5252740,0.2499956,-2.14148296,-1.22079085,-0.4807605,1.42696349,⋯,-1.1394080,-0.9583440,-0.32305449,-1.1777979,-0.9938126,-0.21822492,-0.3883285,-1.2838415,Na+/K+ pump inhibitor,Small Molecule


In [19]:
mat <- organoid_consensus_df %>%
  select(-patient, -treatment, -dose, -unite, -Target, -Class,) %>%
  column_to_rownames("profile_id") %>%
  as.matrix()
  

ERROR: Error in column_to_rownames(., "profile_id"): could not find function "column_to_rownames"


In [ ]:
 heatmap_plot <- Heatmap(
            mat, 
            col = col_fun,
            show_row_names = FALSE,
            cluster_columns = FALSE,
            show_column_names = FALSE,

            column_names_gp = gpar(fontsize = 16), # Column name label formatting
            row_names_gp = gpar(fontsize = 14), 

            show_heatmap_legend = FALSE,
            heatmap_legend_param = list(
                        title = "Feature\nValue",
                        title_position = "topcenter", 
                        title_gp = gpar(fontsize = 16, angle = 0, fontface = "bold", hjust = 1.0),
                        labels_gp = gpar(fontsize = 16),
                        legend_height = unit(4, "cm"),
                        legend_width = unit(3, "cm"),
                        annotation_legend_side = "bottom"
                        ), 
            row_dend_width = unit(2, "cm"),
            column_title = paste0("Dose: ", dose," uM"),
            top_annotation = column_anno,
        )
    }